In [1]:
import geopy
from geopy.geocoders import Nominatim
import requests
import folium
import pandas as pd
from time import sleep
from copy import deepcopy

pd.options.mode.chained_assignment = None

## LOAD DATA

In [2]:
with open('cast_types.txt', 'r') as file:
    cast_types = [f.strip('\n') for f in file.readlines()]

In [3]:
data = {}
for cast_ in cast_types:
    data[cast_] = pd.read_csv(f'data/{cast_}.csv').set_index('name')

## DIRECTIONS API

In [4]:
with open('gitignore/api_key.txt') as fh:
    api_key = fh.read()

In [5]:
geolocator = Nominatim(user_agent="Transport_planner")

## CREATE OBJECT

In [6]:
class TransportPlanner(object):
    
    def __init__(self, supervisors, drivers, cast_members, destinations, geolocator=geolocator, api_key=api_key):
        self.supervisors = supervisors
        self.drivers = drivers
        self.cast_members = cast_members
        self.destinations = destinations
        self.all_locs = [self.supervisors, self.drivers, self.cast_members, self.destinations]
        self.geolocator = geolocator
        self.api_key = api_key
        self.capacities = sorted(list(set(tp.drivers['capacity'])))
        with open('cast_types.txt', 'r') as file:
            self.cast_types = [f.strip('\n') for f in file.readlines()] 
        
    def __str__(self):
        return('This is a transport planner object.')
    
    
    def get_locations(self):
        self.locations = {}
        for df in self.all_locs:
            for ind, row in df.iterrows():
                try:
                    self.locations[ind] = self.geolocator.geocode(row['address'])
                except:
                    continue
    
    def get_coordinates(self):
        self.coords = pd.DataFrame({k: (v.longitude, v.latitude) for k, v in self.locations.items()}).T
    
    def get_directions_locations(self, loc_1, loc_2):
        # get direction as a geojson
        url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={self.api_key}&start={str(loc_1.longitude)},{str(loc_1.latitude)}&end={str(loc_2.longitude)},{str(loc_2.latitude)}"
        r = requests.get(url)
        return r.json()
    
    def get_directions_coords(self, coords_1, coords_2):
        # get direction as a geojson
        url = f"https://api.openrouteservice.org/v2/directions/driving-car?api_key={self.api_key}&start={str(coords_1[0])},{str(coords_1[1])}&end={str(coords_2[0])},{str(coords_2[1])}"
        r = requests.get(url)
        return r.json()
    
    def get_duration(self, loc_1, loc_2, loc=True):
        # return time of route plan in seconds
        if loc:
            geojson = self.get_directions_locations(loc_1, loc_2)
        else:
            geojson = self.get_directions_coords(loc_1, loc_2)
        return geojson, geojson['features'][0]['properties']['segments'][0]['duration']
    
    def get_distance_matrix(self):
        dist_table = {}
        geojsons = {}
        counter = 0
        for key, loc in self.locations.items():
            dist_table[key] = {}
            geojsons[key] = {}
            for keyy, locc in self.locations.items():
                
                # for rate limit
                counter += 1
                if counter > 39:
                    sleep(60)
                    counter = 0
                else:
                    pass
                
                # get durations
                if key == keyy:
                    dur = 0
                else:
                    geojson, dur = self.get_duration(loc, locc, True)
                geojsons[key][keyy] = geojson
                dist_table[key][keyy] = dur
                
        self.dist_table = dist_table
        self.geojsons = geojsons
        return None

In [7]:
tp = TransportPlanner(*[data[f] for f in cast_types])

In [8]:
tp.get_locations()

In [9]:
tp.get_coordinates()

In [10]:
tp.get_distance_matrix()

{'Laci': {'Laci': 0,
  'Feri': 1053.2,
  'Dani': 264.3,
  'Deni': 530.8,
  'Bius': 237.9,
  'Zoli': 132.0,
  'Peti': 840.4,
  'Nori': 837.9,
  'Boci': 729.9,
  'Coci': 399.2,
  'Loci': 254.6,
  'Apci': 413.8,
  'Shooting': 1197.4,
  'Dressing': 444.0},
 'Feri': {'Laci': 1129.2,
  'Feri': 0,
  'Dani': 946.8,
  'Deni': 1399.5,
  'Bius': 868.3,
  'Zoli': 970.8,
  'Peti': 437.5,
  'Nori': 610.1,
  'Boci': 848.6,
  'Coci': 1261.0,
  'Loci': 1228.4,
  'Apci': 1010.7,
  'Shooting': 1788.2,
  'Dressing': 1417.9},
 'Dani': {'Laci': 270.4,
  'Feri': 936.8,
  'Dani': 0,
  'Deni': 525.3,
  'Bius': 176.8,
  'Zoli': 116.2,
  'Peti': 724.0,
  'Nori': 723.1,
  'Boci': 729.9,
  'Coci': 518.5,
  'Loci': 373.9,
  'Apci': 297.5,
  'Shooting': 1080.5,
  'Dressing': 563.3},
 'Deni': {'Laci': 575.4,
  'Feri': 1406.0,
  'Dani': 497.4,
  'Deni': 0,
  'Bius': 590.7,
  'Zoli': 496.9,
  'Peti': 1193.2,
  'Nori': 1192.2,
  'Boci': 1094.9,
  'Coci': 796.8,
  'Loci': 461.1,
  'Apci': 669.2,
  'Shooting': 836.0,
  'D

In [11]:
tp.dist_table['Feri']['Laci']

1129.2

In [14]:
dest = tp.destinations.index[0]

In [131]:
def optimize_destination(dest):
    cast_done = []
    drivers = {}
    

def get_optimal_route(dest, capacity, cast_done):
    driving_order = [dest]
    cast_solved = []
    start = dest
    relevant_cast_names = tp.cast_members[tp.cast_members['destination'] == dest].index
    relevant_cast_names = [f for f in relevant_cast_names if not f in cast_done]
    while len(driving_order) < capacity + 1:
        if len(relevant_cast_names) == 0:
            return driving_order
        next_cast = get_next_cast_member(start, relevant_cast_names)
        driving_order.append(next_cast)
        relevant_cast_names = [f for f in relevant_cast_names if not f in driving_order]
        start = next_cast
    return driving_order
    
def get_next_cast_member(start, relevant_cast_names):
    relevant_dists = {k: v for k, v in tp.dist_table[start].items() if k in relevant_cast_names}
    return min(relevant_dists, key=relevant_dists.get)

In [140]:
## itt kéne megoldani, hogy egy helyszínt fullba megcsináljunk, vezetőket allokáljunk

cast_done = []
drivers = {}
relevant_cast = tp.cast_members[tp.cast_members['destination'] == dest].index
#while cast_done

In [141]:
relevant_cast

Index(['Bius', 'Peti', 'Nori', 'Loci', 'Apci'], dtype='object', name='name')

In [132]:
ut = get_optimal_route('Shooting', 3, [])
ut

['Shooting', 'Apci', 'Bius', 'Loci']

In [133]:
ut2 = get_optimal_route('Shooting', 3, ut)

In [134]:
ut2

['Shooting', 'Nori', 'Peti']

In [119]:
tp.cast_members.loc[ut]

C:\Users\kapronczaym\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


,address,destination,mailto
name,,,
Shooting,NaN,NaN,NaN
Apci,"Budapest, Szent István körút 14, 1137",Shooting,mor.kapronczay@gmail.com
Bius,1077 Budapest Wesselényi utca 58.,Shooting,mor.kapronczay@gmail.com
Loci,Lónyay u. 13 Budapest 1093,Shooting,mor.kapronczay@gmail.com
